<a href="https://colab.research.google.com/github/Ron-Wu/Machine-learning-0602/blob/main/titanic1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from urllib.request import urlretrieve
url = "https://github.com/Ron-Wu/Machine-learning-0602/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Ron-Wu/Machine-learning-0602/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f579ce4b710>)

In [39]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding = "utf-8")
test_df = pd.read_csv("test.csv", encoding = "utf-8")

In [40]:
# train_df  # 測試看是否有成功列出訓練資料
# test_df  # 測試看是否有成功列出測試資料

In [41]:
data = pd.concat([train_df, test_df], ignore_index = True)
data = data.drop(["PassengerId", "Survived"], axis = 1)

# data = pd.concat([train_df, test_df], ignore_index=True)
# data = data.drop(["PassengerId", "Survived"], axis=1)

In [42]:
na = data.isna().sum()
# series[帶入跟你的資料筆數一樣多True/False list]
na[na > 0].sort_values()
# na[na > 0].sort_values(ascending=False)  ## ascending=False 讓資料倒序

Fare           1
Embarked       2
Age          263
Cabin       1014
dtype: int64

In [43]:
def cabin_head(s):
  if not pd.isna(s):
    return s[0]
data["Cabin"] = data["Cabin"].apply(cabin_head)

In [44]:
dic = data["Ticket"].value_counts()
data["Ticket"] = data["Ticket"].apply(lambda t:dic[t])

In [45]:
# 補缺失值(類別：最常出現) [!如果一堆測試資料，不要重算，直接補Ｓ]
most = data["Embarked"].value_counts().idxmax() ## .idxmax()：取得最大值的索引，.idxmin()：取得最小值的索引
data["Embarked"] = data["Embarked"].fillna(most)
na = data.isna().sum()
na[na > 0].sort_values()

Fare        1
Age       263
Cabin    1014
dtype: int64

In [46]:
# 補缺失值，數值：補中位數
med = data.median().drop(["Pclass"])  # 因為Pclass非數字型態，所以要在中位數字中將其先去除
data = data.fillna(med)
na = data.isna().sum()
na[na > 0].sort_values()

Cabin    1014
dtype: int64

In [47]:
def name_convert(s):
  s = s.split(",")[-1].split(".")[0]
  s = s.strip()
  return s
counts = data["Name"].apply(name_convert).value_counts()
whitelist = counts[counts > 50].index
def name_convert(s):
  s = s.split(",")[-1].split(".")[0]
  s = s.strip()
  if s in whitelist:
    return s
  else: 
    return None
  
data["Name"] = data["Name"].apply(name_convert)

In [48]:
data = pd.get_dummies(data)
data = pd.get_dummies(data, columns=["Pclass"])
data

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [49]:
# 模型建立完，後續的嘗試，只是老師嘗試過，所以直接做
data["Family"] = data["SibSp"] + data["Parch"]
data


,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [50]:
# len(data)
# data.shape # 可取得一個tuple含有欄數和列數，例(1309, 26)

In [51]:
# .loc(根據列編號)(X)     .iloc(根據第幾個)(O)
# .iloc [第一列, 第二列, 第三列...]

# 此步驟將先前的資料拆分回來
x_train = data.iloc[:train_df.shape[0]]
y_train = train_df["Survived"]
x_predict = data.iloc[train_df.shape[0]:]
# x_train
# x_predict

In [52]:
# import numpy as np
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import cross_val_score
# from  sklearn.ensemble import RandomForestClassifier
# params = {
#     # 1. 5, 2. [1, 2, 3], 3. range
#     # 20~99
#     "n_estimators" : range(20, 100),
#     # 3~10
#     "max_depth" : range(3, 11)
# }
# clf = RandomForestClassifier()
# cv = GridSearchCV(clf, params, cv = 10, n_jobs = -1)
# cv.fit(x_train, y_train)
# print(cv.best_score_)
# print(cv.best_params_)


import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
params = {
    # 1. 5 2. [1, 2, 3] 3. range
    # 20~99
    "n_estimators":range(20, 100),
    # 3~10
    "max_depth":range(3, 11)
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train, y_train)
print(cv.best_score_)
print(cv.best_params_)

0.847365792759051
{'max_depth': 7, 'n_estimators': 97}


In [58]:
# clf = RandomForestClassifier(n_estimators=25, max_depth=6)
# scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=-1)
# print("10:", scores)
# print("average:", np.average(scores))

clf = RandomForestClassifier(n_estimators=97, max_depth=7)
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=-1)
print("10:", scores)
print("average:", np.average(scores))

10: [0.81111111 0.85393258 0.74157303 0.8988764  0.86516854 0.84269663
 0.84269663 0.78651685 0.85393258 0.82022472]
average: 0.8316729088639201


In [59]:
clf = RandomForestClassifier(n_estimators=97, max_depth=7)
clf.fit(x_train, y_train)
pre = clf.predict(x_predict)
df = pd.DataFrame({
    "PassengerId" : test_df["PassengerId"],
    "Survived" : pre
})
df.to_csv("rf.csv", encoding = "utf-8", index = False)
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [61]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# data_scale = scaler.fit_transform(data)
# data_scale = pd.DataFrame(data_scale, columns=data.columns)
# # .loc (根據列編號)(X) .iloc (根據第幾個)(O)
# # .iloc [第一列, 第二列, 第三列...]
# x_train_scale = data_scale.iloc[:train_df.shape[0]]
# x_predict_scale = data_scale.iloc[train_df.shape[0]:]
# x_train_scale

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scale = scaler.fit_transform(data)
data_scale = pd.DataFrame(data_scale, columns=data.columns)
# .loc (根據列編號)(X) .iloc (根據第幾個)(O)
# .iloc [第一列, 第二列, 第三列...]
x_train_scale = data_scale.iloc[:train_df.shape[0]]
x_predict_scale = data_scale.iloc[train_df.shape[0]:]
x_train_scale

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,0.273456,0.125,0.000000,0.0,0.014151,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.1
1,0.473882,0.125,0.000000,0.1,0.139136,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.1
2,0.323563,0.000,0.000000,0.0,0.015469,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.436302,0.125,0.000000,0.1,0.103644,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.1
4,0.436302,0.000,0.000000,0.0,0.015713,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.336089,0.000,0.000000,0.0,0.025374,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
887,0.235876,0.000,0.000000,0.0,0.058556,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
888,0.348616,0.125,0.222222,0.3,0.045771,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.3
889,0.323563,0.000,0.000000,0.0,0.058556,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [62]:
# from sklearn.neighbors import KNeighborsClassifier
# params = {
#     "n_neighbors":range(3, 100)
# }
# clf = KNeighborsClassifier()
# cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
# cv.fit(x_train_scale, y_train)
# print(cv.best_score_)
# print(cv.best_params_)

from sklearn.neighbors import KNeighborsClassifier
params = {
    "n_neighbors":range(3, 100),
}
clf = KNeighborsClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train, y_train)
print(cv.best_score_)
print(cv.best_params_)

0.7397128589263421
{'n_neighbors': 5}


In [63]:
# clf = KNeighborsClassifier(n_neighbors=11)
# clf.fit(x_train_scale, y_train)
# pre = clf.predict(x_predict_scale)
# df = pd.DataFrame({
#     "PassengerId":test_df["PassengerId"],
#     "Survived":pre
# })
# df.to_csv("knn.csv", encoding="utf-8", index=False)
# df

clf = KNeighborsClassifier(n_neighbors=11)
clf.fit(x_train_scale, y_train)
pre = clf.predict(x_predict_scale)
df = pd.DataFrame({
    "PassengerId" : test_df["PassengerId"],
    "Survived" : pre
})
df.to_csv("knn.csv", encoding = "utf-8", index = False)
df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
